# Portfolio Optimization using Modern Portfolio Theory

### Harry Markowitz theorized that we can construct an 'efficient frontier' of optimal portfolios. An asset's risk and return shouldn't be viewed in isolation; it should be viewed in the context of a portfolio's overall performance. The frontier is a curve wherein given a certain risk, we can maximize the expected return. 

In [109]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import random

random.seed(3456)
plotly.tools.set_credentials_file(username='amurali8', api_key='gkGQLcSnSFYgyUoK8QjB')

In [91]:
#Creating list of tickers
tickers = ['BEP','AAPL','WMT','PEP','ACN','GOOGL'];

#BEP - Brookfield Renewable Partners
#AAPL - Apple
#WMT - Walmart
#PEP - PepsiCo
#ACN - Accenture
#GOOGL - Alphabet

In [92]:
#Extracting adjusted close data for each of the stock tickers
df = pd.DataFrame();
for t in tickers:
    df[t] = wb.DataReader(t, data_source='yahoo', start = '01-01-2010')['Adj Close'];

In [93]:
#Simple daily return rate
ret = df.pct_change()

In [94]:
#Mean daily return rate and covariance matrix
mean_daily_ret = ret.mean()
mean_daily_ret = np.array(mean_daily_ret)
covariance_matrix = ret.cov()

In [95]:
#Number of portfolios considered
n = 200000;

In [96]:
#Initializing weights
weights = np.zeros((np.size(tickers),1))

In [97]:
#Computing annualized portfolio return and volatility
data_list = [];
weight_data = np.zeros((n,np.size(tickers)))
risk_free_return_rate = 2.07/100;

for i in range(n):
    weights = np.random.random((np.size(tickers),1));
    weights = weights/weights.sum();
    weight_data[i,:] = np.transpose(weights);
    
    #Annual portfolio return
    portfolio_return = np.dot(np.transpose(weights),mean_daily_ret)*252;
    
    #Annual portfolio volatility
    portfolio_std = np.sqrt(np.dot(np.transpose(weights),np.dot(covariance_matrix,weights))*252);
    
    #Sharpe ratio = return/volatility
    sharpe = (portfolio_return-risk_free_return_rate)/portfolio_std
    
    temp = [float(portfolio_return), float(portfolio_std), float(sharpe)];
    
    data_list.append(temp)

data_rvs = pd.DataFrame(data_list,columns = ['Portfolio_Return','Portfolio_Volatility','Sharpe_Ratio'])
weight_data = pd.DataFrame(weight_data,columns = tickers)

In [98]:
#Identifying portfolios with maximum sharpe ratio and minimum volatility
max_sharpe = weight_data.iloc[data_rvs['Sharpe_Ratio'].idxmax()]
min_vol = weight_data.iloc[data_rvs['Portfolio_Volatility'].idxmin()]

In [99]:
#Finding return, volatility, and sharpe ratio of these optimized portfolios
temp1 = [];
temp2 = [];
for i in range(n):
    if weight_data.iloc[i].tolist() == max_sharpe.tolist():
        temp1 = data_rvs.iloc[i];
    elif weight_data.iloc[i].tolist() == min_vol.tolist():
        temp2 = data_rvs.iloc[i];

max_sharpe_rvs = temp1;
min_vol_rvs = temp2;

In [110]:
#Plotting the efficient frontier

trace = go.Scattergl(x = data_rvs['Portfolio_Volatility'], y = data_rvs['Portfolio_Return'], mode = 'markers', 
                   marker = dict (size = 14, color = data_rvs['Sharpe_Ratio'], colorscale = 'rainbow', showscale = True)
                  )
data = [trace]

layout = go.Layout(
    
    title=go.layout.Title(
        text=r'$\text{Efficient Frontier}$'
    ),
    xaxis=go.layout.XAxis(
        title=r'$\text{Volatility}$'
    ),
    yaxis=go.layout.YAxis(
        title = r'$\text{Expected Return}$'
    )
        
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='efficient_frontier')


In [103]:
print('Max Sharpe Ratio Portfolio (%):'); print('\v' + '\n' + 'Portfolio Return: ' + str(round(max_sharpe_rvs[0]*100,6)) + '\n' + 'Portfolio Volatility: ' + str(round(max_sharpe_rvs[1]*100,6)) + '\n' + 'Sharpe Ratio: ' + str(round(max_sharpe_rvs[2],6)) + ' (dimensionless)'); print( '\n'+ str(max_sharpe*100))

Max Sharpe Ratio Portfolio (%):

Portfolio Return: 17.642134
Portfolio Volatility: 13.727081
Sharpe Ratio: 1.13441 (dimensionless)

BEP      15.239122
AAPL     25.271658
WMT      11.088408
PEP      26.184847
ACN      22.160381
GOOGL     0.055584
Name: 60580, dtype: float64


In [104]:
print('Min Volatility Portfolio (%):'); print( '\n' + 'Portfolio Return: ' + str(round(min_vol_rvs[0]*100,6)) + '\n' + 'Portfolio Volatility: ' + str(round(min_vol_rvs[1]*100,6)) + '\n' + 'Sharpe Ratio: ' + str(round(min_vol_rvs[2],6)) + ' (dimensionless)'); print('\n'+ str(min_vol*100))

Min Volatility Portfolio (%):

Portfolio Return: 14.079245
Portfolio Volatility: 12.009207
Sharpe Ratio: 1.000003 (dimensionless)

BEP      15.142377
AAPL      7.707261
WMT      26.969167
PEP      42.146449
ACN       3.165033
GOOGL     4.869712
Name: 141211, dtype: float64
